In [ ]:
"""Tutorial: Symmetry-Adapted Perturbation Theory (SAPT0)"""

__author__    = ["Daniel G. A. Smith", "Konrad Patkowski"]
__credit__    = ["Daniel G. A. Smith", "Konrad Patkowski"]

__copyright__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2017-06-24"

# Symmetry-Adapted Perturbation Theory in Atomic Orbitals (SAPT0-AO)

In this tutorial, we revisit the SAPT0 calculation of interaction energy in the water dimer from the previous example. If you have not yet examined the introductory SAPT0 tutorial, this example might be very confusing.
The new element here is that all exchange corrections $E^{(100)}_{\rm exch}$, $E^{(200)}_{\rm exch-ind,resp}$, and $E^{(200)}_{\rm exch-disp}$, as well as the electrostatic energy $E^{(100)}_{\rm elst}$, are computed in the atomic-orbital (AO) basis rather than the conventional molecular-orbital (MO) one. This should not affect the results at all: in fact, we expect exactly the same numerical answers as in the previous example (note that we are using exactly the same level of theory, including the $S^2$ approximation in all exchange corrections).
Why recast the SAPT0 expressions from MOs into AOs? The reasons we give here explain why the AO-based formalism has the *potential* to be more computationally efficient. It does not mean that the AO approach as implemented in this example is already more efficient than the MO one - in general, it is not! (Not that it really matters as this example takes just a few seconds to run).
1. There is no need for computing and storing all different kinds of SAPT0 integrals in the MO basis. In this specific example, the only MO integrals needed are the ones required for the dispersion amplitudes $t^{rs}_{ab}$ and CPHF coefficients (relaxed induction amplitudes) $C^r_a$,$C^s_b$, that is, the quantities involving excitation energy denominators. Once formed, these amplitudes will be back-transformed into the AO basis.
2. Most of the tensor contractions involving two-electron integrals (naturally, these are the most expensive contractions present) can be formulated in the language of *generalized Coulomb and exchange (JK) operators* (see below). In fact, the entire $E^{(100)}_{\rm exch}$ and $E^{(200)}_{\rm exch-ind,resp}$ terms can be implemented in this way (but not $E^{(200)}_{\rm exch-disp}$). The developers of Psi4 have put a lot of effort into optimizing the computation of generalized JK matrices and, as a rule of thumb, if your Psi4 algorithm can utilize generalized JK matrices, it will be a very good idea to do so.
3. The AO algorithm can maximize the computational savings when density-fitted integrals and intermediates are employed (however, this specific example does not use any density fitting).

The original formulation of SAPT involves only MOs, and the AO formulation is not very well documented in the SAPT literature. Therefore, we will explicitly list all the formulas for the SAPT0 corrections computed below - some of these expressions are nowhere to be found in the published papers! A limited set of AO expressions for closed-shell SAPT0 is given in [Hesselmann:2005], and the analogous formulas for open-shell, UHF-based SAPT were first reported in [Hapka:2012]. A corrected and extended set of AO SAPT(UHF) expressions can be found in the supplementary material to [Gonthier:2016], however, even this reference does not have everything that we need (the exchange-dispersion energy is only formulated with density fitting). Therefore, please pay close attention to the expressions that need to be implemented.


In [ ]:
# A simple Psi 4 input script to compute SAPT interaction energies
# All exchange corrections are computed using AO algorithms
#
# Created by: Konrad Patkowski
# Helper routines by: Daniel G. A. Smith
# Date: 6/8/17
# License: GPL v3.0
#

import time
import numpy as np
from helper_SAPT import *
np.set_printoptions(precision=5, linewidth=200, threshold=2000, suppress=True)
import psi4

# Set Psi4 & NumPy Memory Options
psi4.set_memory('2 GB')
psi4.core.set_output_file('output.dat', False)

numpy_memory = 2

# Set molecule to dimer
dimer = psi4.geometry("""
O   -0.066999140   0.000000000   1.494354740
H    0.815734270   0.000000000   1.865866390
H    0.068855100   0.000000000   0.539142770
--
O    0.062547750   0.000000000  -1.422632080
H   -0.406965400  -0.760178410  -1.771744500
H   -0.406965400   0.760178410  -1.771744500
symmetry c1
""")

psi4.set_options({'basis': 'jun-cc-pVDZ',
                  'e_convergence': 1e-8,
                  'd_convergence': 1e-8})

sapt = helper_SAPT(dimer, memory=8, algorithm='AO')


# 1. Preparation of the matrix elements

Note that, similar to the previous example, we have performed HF calculations for both monomers and initialized the pertinent integrals and intermediates via a call to `helper_SAPT`. This time, we set the additional optional parameter `algorithm` to `'AO'` (the default is `'MO'`) so that the generalized JK matrix calculations are set up and ready to go.

It is now time to prepare AO-based intermediates and introduce the pertinent notation. With the capital letters $K,L,\ldots$ denoting the AO basis (for simplicity, we assume that this basis is the same for monomers A and B), the matrix $C_{Ka}$ (`Ci`) denotes the SCF vectors for the occupied orbitals on A, and matrices `Cj`, `Cr`, and `Cs` collect the similar vectors for occupied orbitals on B, virtual orbitals on A, and virtual orbitals on B, respectively. The quantity $P^{\rm A}_{KL}=C_{Ka}C_{La}$ (denoted in the code by `Pi`) is one half of the monomer-A density matrix in the AO basis: similarly, $P^{\rm B}$ (`Pj`) is one half of the AO density matrix for monomer B. `S` denotes the overlap matrix in the AO basis, and `I` is the two-electron integral matrix $(KL|MN)$ (in the (11|22) index order) taken straight from `mints.ao_eri()`.
The key intermediates in AO-based SAPT, the generalized Coulomb and exchange matrices, exhibit highly inconsistent notation in the SAPT literature. The definitions from [Hesselmann:2005] and [Hapka:2012] differ by a factor of 2: we will follow [Hapka:2012] and define these matrices as

\begin{equation}
{J}[{X}]_{KL} = (KL|MN) {X}_{MN}
\end{equation}

\begin{equation}
{K}[{X}]_{KL} = (KM|NL) {X}_{MN}
\end{equation}

An alternative definition is employed in [Gonthier:2016] and involves an explicit summation over occupied orbitals (therefore, we have to specify explicitly the monomer A or B). The generalized JK matrices defined in this way (for any two matrices $A$,$B$) will be denoted by $\bar{J}$ and $\bar{K}$, respectively:

\begin{equation}
\bar{J}^{\rm A}[{A,B}]_{KL} = (KL|MN) {A}_{Ma}{B}_{Na} 
\end{equation}

\begin{equation}
\bar{K}^{\rm A}[{A,B}]_{KL} = (KM|NL) {A}_{Ma}{B}_{Na} 
\end{equation}

and the monomer-B quantities $\bar{J}^{\rm B}[{A,B}]$ and $\bar{K}^{\rm B}[{A,B}]$ are defined with the summation over $a$ replaced by the one over $b$. The generalized JK matrices in either notation reduce to the ordinary JK matrices as follows:

\begin{equation}
J^{\rm A}\equiv J[P^{\rm A}]\equiv \bar{J}^{\rm A}[Ci,Ci] 
\end{equation}

and the same for $K$. Going back to the notation of [Hapka:2012], the intermediates needed for SAPT-AO are both standard JK matrices for both monomers `(Jii, Kii, Jjj, Kjj)` and generalized matrices $J[P^{\rm A}S P^{\rm B}]$, $K[P^{\rm A}S P^{\rm B}]$ `(Jij,Kij)`. Note that $J[P^{\rm B}S P^{\rm A}]=J[P^{\rm A}S P^{\rm B}]$ and $K[P^{\rm B}S P^{\rm A}]=K[P^{\rm A}S P^{\rm B}]^T$, so it is sufficient to generate the JK matrices for just one of these two operators.
The last intermediates that we need are the matrices of the monomer electrostatic potential $\omega^{\rm X}=v^{\rm X}+2J^{\rm X}$ and of the complete monomer Fock operator $F^{\rm X}=v^{\rm X}+2J^{\rm X}-K^{\rm X}$ for X=A,B, where $v^{\rm X}$ is the matrix of the nuclear attraction operator. In the code below, these matrices are stored as `w_A, w_B, h_A, h_B`.


In [ ]:
# Build intermediates
int_timer = sapt_timer('intermediates')
Pi = np.dot(sapt.orbitals['a'], sapt.orbitals['a'].T)
Pj = np.dot(sapt.orbitals['b'], sapt.orbitals['b'].T)

S = sapt.S
num_el_A = (2 * sapt.ndocc_A)
num_el_B = (2 * sapt.ndocc_B)

Ci = sapt.orbitals['a']
Cj = sapt.orbitals['b']
Cr = sapt.orbitals['r']
Cs = sapt.orbitals['s']

I = np.asarray(sapt.mints.ao_eri())

Jii, Kii = sapt.compute_sapt_JK(Ci, Ci)
Jjj, Kjj = sapt.compute_sapt_JK(Cj, Cj)

Jij, Kij = sapt.compute_sapt_JK(Ci, Cj, tensor=sapt.chain_dot(Ci.T, S, Cj))

w_A = sapt.V_A + 2 * Jii
w_B = sapt.V_B + 2 * Jjj

h_A = sapt.V_A + 2 * Jii - Kii
h_B = sapt.V_B + 2 * Jjj - Kjj

int_timer.stop()



# 2. Electrostatic energy

In the AO formalism, the SAPT0 electrostatic energy $E^{(100)}_{\rm elst}$ is given by

\begin{equation}
E^{(100)}_{\rm elst}=4P^{\rm A}\cdot J^{\rm B} + 2P^{\rm A}\cdot v^{\rm B} + 2P^{\rm B}\cdot v^{\rm A} + V_{\rm nuc},
\end{equation}

where $V_{\rm nuc}$ is the intermolecular nuclear repulsion energy. We define the dot-product notation for two matrices as follows,

\begin{equation}
A\cdot B = A_{KL}B_{KL}
\end{equation}



In [ ]:
### Build electrostatics
elst_timer = sapt_timer('electrostatics')
two_el = 2 * np.vdot(Pi, Jjj)
att_a = np.vdot(sapt.V_A, Pj)
att_b = np.vdot(sapt.V_B, Pi)
rep = sapt.nuc_rep
elst_ijij = 2 * (two_el + att_a + att_b) + rep

Elst10 = elst_ijij
sapt_printer('Elst10', Elst10)
elst_timer.stop()
### End electrostatics



# 3. First-order exchange energy

The SAPT0 first-order exchange energy $E^{(100)}_{\rm exch}$ within the $S^2$ approximation and the density matrix formalism [Moszynski:1994b] can be recast into AOs as follows,

\begin{align}
E^{(100)}_{\rm exch}=&- 2P^{\rm B}\cdot K^{\rm A} -2 (P^{\rm A}SP^{\rm B})\cdot (F^{\rm A}+F^{\rm B})\\ & +2 (P^{\rm B}SP^{\rm A}SP^{\rm B})\cdot \omega^{\rm A}+2 (P^{\rm A}SP^{\rm B}SP^{\rm A})\cdot \omega^{\rm B}\\ &-2 (P^{\rm A}SP^{\rm B})\cdot K[P^{\rm A}SP^{\rm B}]
\end{align}

In the implementation below, the `sapt.chain_dot()` call is nothing more than a multiplication of a chain of matrices - a series of `np.dot()` calls.


In [ ]:
### Start exchange
exch_timer = sapt_timer('exchange')
exch = 0
exch -= 2 * np.vdot(Pj, Kii)
exch -= 2 * np.vdot(sapt.chain_dot(Pi, S, Pj), (h_A + h_B))

exch += 2 * np.vdot(sapt.chain_dot(Pj, S, Pi, S, Pj), w_A)
exch += 2 * np.vdot(sapt.chain_dot(Pi, S, Pj, S, Pi), w_B)

exch -= 2 * np.vdot(sapt.chain_dot(Pi, S, Pj), Kij)

Exch100 = exch
sapt_printer('Exch10(S^2)', Exch100)
exch_timer.stop()
### End E100 (S^2) Exchange



# 4. Dispersion energy

As the SAPT0 dispersion energy $E^{(200)}_{\rm disp}$ involves an energy denominator, it is calculated in the MO formalism in exactly the same way as in the previous example:

\begin{equation}
E^{(200)}_{\rm disp}=4t^{rs}_{ab}v^{ab}_{rs}\;\;\;\;\;\; t^{rs}_{ab}=\frac{v_{ab}^{rs}}{\epsilon_a+\epsilon_b-\epsilon_r-\epsilon_s}
\end{equation}



In [ ]:
### Start E200 Disp
disp_timer = sapt_timer('dispersion')
v_abrs = sapt.v('abrs')
v_rsab = sapt.v('rsab')
e_rsab = 1/(-sapt.eps('r', dim=4) - sapt.eps('s', dim=3) + sapt.eps('a', dim=2) + sapt.eps('b'))

Disp200 = 4 * np.einsum('rsab,rsab,abrs->', e_rsab, v_rsab, v_abrs)
sapt_printer('Disp20', Disp200)
### End E200 Disp



# 5. Exchange dispersion energy

In order to compute the SAPT0 exchange-dispersion energy $E^{(200)}_{\rm exch-disp}$ using AO-based quantities, the dispersion amplitude $t^{rs}_{ab}$ needs to be backtransformed into the AO basis. Note that the resulting AO amplitude has no index symmetry whatsoever.

\begin{equation}
t_{KM}^{LN}=t_{ab}^{rs} C_{Ka} C_{Mb} C_{Lr} C_{Ns}
\end{equation}

The SAPT0-AO exchange dispersion energy is now given by

\begin{align}
E^{(200)}_{\rm exch-disp} = & t_{KM}^{LN} \left[-2 (KN|ML)-2 S_{KN} (F^{\rm A})_{ML}-2 S_{ML} (F^{\rm B})_{KN} \right. \\ &
-4 (KL|MQ) (SP^{\rm A})_{NQ}+2 (ML|KQ) (SP^{\rm A})_{NQ}-4 (MN|KQ) (SP^{\rm B})_{LQ}+2 (KN|MQ) (SP^{\rm B})_{LQ} \\ &
-4 (\omega^{\rm A})_{MN} (SP^{\rm B}S)_{KL}+2 S_{KN} (\omega^{\rm A} P^{\rm B}S)_{ML}+2 S_{ML} (\omega^{\rm A} P^{\rm B}S)_{NK} \\ &
-4 (\omega^{\rm B})_{KL} (SP^{\rm A}S)_{MN}+2 S_{ML} (\omega^{\rm B} P^{\rm A}S)_{KN}+2 S_{KN} (\omega^{\rm B} P^{\rm A}S)_{LM} \\ &
+4 (KQ|MN) (SP^{\rm B}SP^{\rm A})_{LQ}+4 (PL|MN) (SP^{\rm B}SP^{\rm A})_{KP}+4 (KL|MS) (SP^{\rm A}SP^{\rm B})_{NS}+4 (KL|RN) (SP^{\rm A}SP^{\rm B})_{MR} \\ &
\left. -2 S_{KN} K[P^{\rm B}SP^{\rm A}]_{ML}-2 S_{ML} K[P^{\rm B}SP^{\rm A}]_{NK}-2 (MS|KQ) (SP^{\rm B})_{LS} (SP^{\rm A})_{NQ}-2 (NR|LP) (SP^{\rm B})_{KR} (SP^{\rm A})_{MP}\right] 
\end{align}



In [ ]:
### Start E200 Exchange-Dispersion

# Build t_rsab
t_rsab = np.einsum('rsab,rsab->rsab', v_rsab, e_rsab)

#backtransform t_rsab to the AO basis
t_lsab = np.einsum('rsab,rl->lsab', t_rsab, Cr.T)
t_lnab = np.einsum('lsab,sn->lnab', t_lsab, Cs.T)
t_lnkb = np.einsum('lnab,ak->lnkb', t_lnab, Ci.T)
t_lnkm = np.einsum('lnkb,bm->lnkm', t_lnkb, Cj.T)

ExchDisp20 = - 2 * np.einsum('lnkm,knml->', t_lnkm, I)

ExchDisp20 -= 2 * np.einsum('lnkm,ml,kn->', t_lnkm, h_A, S)
ExchDisp20 -= 2 * np.einsum('lnkm,ml,kn->', t_lnkm, S, h_B)

interm = 2 * np.einsum('klmq,nq->klmn', I, np.dot(S, Pi))
ExchDisp20 -= 2 * np.einsum('lnkm,klmn->', t_lnkm, interm)
ExchDisp20 += np.einsum('lnkm,mlkn->', t_lnkm, interm)

interm = 2 * np.einsum('klmq,nq->klmn', I, np.dot(S, Pj))
ExchDisp20 -= 2 * np.einsum('lnkm,mnkl->', t_lnkm, interm)
ExchDisp20 += np.einsum('lnkm,knml->', t_lnkm, interm)

ExchDisp20 -= 4 * np.einsum('lnkm,mn,kl->', t_lnkm, w_A, sapt.chain_dot(S, Pj, S))
ExchDisp20 += 2 * np.einsum('lnkm,kn,ml->', t_lnkm, S, sapt.chain_dot(w_A, Pj, S))
ExchDisp20 += 2 * np.einsum('lnkm,ml,nk->', t_lnkm, S, sapt.chain_dot(w_A, Pj, S))

ExchDisp20 -= 4 * np.einsum('lnkm,kl,mn->', t_lnkm, w_B, sapt.chain_dot(S, Pi, S))
ExchDisp20 += 2 * np.einsum('lnkm,ml,kn->', t_lnkm, S, sapt.chain_dot(w_B, Pi, S))
ExchDisp20 += 2 * np.einsum('lnkm,kn,lm->', t_lnkm, S, sapt.chain_dot(w_B, Pi, S))

spbspa = sapt.chain_dot(S, Pj, S, Pi)
spaspb = sapt.chain_dot(S, Pi, S, Pj)
interm = np.einsum('kqmn,lq->klmn', I, spbspa)
interm += np.einsum('plmn,kp->klmn', I, spbspa)
interm += np.einsum('klms,ns->klmn', I, spaspb)
interm += np.einsum('klrn,mr->klmn', I, spaspb)
ExchDisp20 += 4 * np.einsum('lnkm,klmn->', t_lnkm, interm)

ExchDisp20 -= 2 * np.einsum('lnkm,kn,ml->', t_lnkm, S, Kij.T)
ExchDisp20 -= 2 * np.einsum('lnkm,ml,nk->', t_lnkm, S, Kij.T)

spa = np.dot(S, Pi)
spb = np.dot(S, Pj)
interm = np.einsum('kpmq,nq->kpmn', I, spa)
interm = np.einsum('kpmn,lp->klmn', interm, spb)
ExchDisp20 -= 2 * np.einsum('lnkm,mlkn->', t_lnkm, interm)
ExchDisp20 -= 2 * np.einsum('lnkm,nklm->', t_lnkm, interm)

sapt_printer('Exch-Disp20', ExchDisp20)
disp_timer.stop()
### End E200 Exchange-Dispersion



# 6. CPHF coefficients and induction energy

The monomer CPHF coefficients $C^a_r$, $C^b_s$ and the SAPT0 induction energy $E^{(200)}_{\rm ind,resp}$ are computed in the MO formalism, identical to the previous example.

\begin{equation}
E^{(200)}_{\rm ind,resp}=4\tilde{v}^{rb}_{ab}C^a_r+4\tilde{v}^{as}_{ab}C^b_s
\end{equation}



In [ ]:
### Start E200 Induction and Exchange-Induction

# E200Induction and CPHF orbitals
ind_timer = sapt_timer('induction')

CPHF_ra, Ind20_ba = sapt.chf('B', ind=True)
sapt_printer('Ind20,r (A<-B)', Ind20_ba)

CPHF_sb, Ind20_ab = sapt.chf('A', ind=True)
sapt_printer('Ind20,r (A->B)', Ind20_ab)

Ind20r = Ind20_ba + Ind20_ab



# 7. Exchange induction energy

We will present the AO formula for $E^{(200)}_{\rm exch-ind,resp}({\rm A}\leftarrow {\rm B})$; the corresponding formula for the B$\leftarrow$A counterpart is obtained by interchanging the quantities (density matrices, JK matrices, $\ldots$) pertaining to A with those of B. We first need to backtransform the CPHF coefficients to the AO basis:

\begin{equation}
C^{\rm CPHF}_{KL}=C^{a}_{r} C_{Ka} C_{Lr} 
\end{equation}

The formula for the A$\leftarrow$B part of $E^{(200)}_{\rm exch-ind,resp}$ now becomes

\begin{align}
E^{(200)}_{\rm exch-ind,resp}({\rm A}\leftarrow {\rm B})=& C^{\rm CPHF}\cdot\left[ 
-2 K^{\rm B}-2 SP^{\rm B}F^{\rm A}-2 F^{\rm B}P^{\rm B}S-4 J[P^{\rm B}SP^{\rm A}] +2 K[P^{\rm A}SP^{\rm B}] \right. \\ & 
+2 \omega^{\rm B}P^{\rm A}SP^{\rm B}S+2 SP^{\rm B}SP^{\rm A}\omega^{\rm B}+2 SP^{\rm B}\omega^{\rm A}P^{\rm B}S \\ & 
\left. +4 J[P^{\rm B}SP^{\rm A}SP^{\rm B}]-2 SP^{\rm B}K[P^{\rm B}SP^{\rm A}]-2 K[P^{\rm A}SP^{\rm B}]P^{\rm B}S\right] 
\end{align}

Note that this correction requires one additional generalized Coulomb matrix, $J[P^{\rm B}SP^{\rm A}SP^{\rm B}]$. In the code below, this matrix is computed and stored in the variable `Jjij`.


In [ ]:
# Exchange-Induction

# A <- B
CPHFA_ao = sapt.chain_dot(Ci, CPHF_ra.T, Cr.T)
ExchInd20_ab = -2 * np.vdot(CPHFA_ao, Kjj)
ExchInd20_ab -= 2 * np.vdot(CPHFA_ao, sapt.chain_dot(S, Pj, h_A))
ExchInd20_ab -= 2 * np.vdot(CPHFA_ao, sapt.chain_dot(h_B, Pj, S))

ExchInd20_ab -= 4 * np.vdot(CPHFA_ao, Jij)
ExchInd20_ab += 2 * np.vdot(CPHFA_ao, Kij)

ExchInd20_ab += 2 * np.vdot(CPHFA_ao, sapt.chain_dot(w_B, Pi, S, Pj, S))
ExchInd20_ab += 2 * np.vdot(CPHFA_ao, sapt.chain_dot(S, Pj, S, Pi, w_B))
ExchInd20_ab += 2 * np.vdot(CPHFA_ao, sapt.chain_dot(S, Pj, w_A, Pj, S))

Jjij, Kjij = sapt.compute_sapt_JK(Cj, Cj, tensor=sapt.chain_dot(Cj.T, S, Pi, S, Cj))

ExchInd20_ab += 4 * np.vdot(CPHFA_ao, Jjij)
ExchInd20_ab -= 2 * np.vdot(CPHFA_ao, sapt.chain_dot(S, Pj, Kij.T))
ExchInd20_ab -= 2 * np.vdot(CPHFA_ao, sapt.chain_dot(Kij, Pj, S))

sapt_printer('Exch-Ind20,r (A<-B)', ExchInd20_ab)

# B <- A
CPHFB_ao = sapt.chain_dot(Cj, CPHF_sb.T, Cs.T)
ExchInd20_ba = -2 * np.vdot(CPHFB_ao, Kii)
ExchInd20_ba -= 2 * np.vdot(CPHFB_ao, sapt.chain_dot(S, Pi, h_B))
ExchInd20_ba -= 2 * np.vdot(CPHFB_ao, sapt.chain_dot(h_A, Pi, S))

ExchInd20_ba -= 4 * np.vdot(CPHFB_ao, Jij)
ExchInd20_ba += 2 * np.vdot(CPHFB_ao, Kij.T)

ExchInd20_ba += 2 * np.vdot(CPHFB_ao, sapt.chain_dot(w_A, Pj, S, Pi, S))
ExchInd20_ba += 2 * np.vdot(CPHFB_ao, sapt.chain_dot(S, Pi, S, Pj, w_A))
ExchInd20_ba += 2 * np.vdot(CPHFB_ao, sapt.chain_dot(S, Pi, w_B, Pi, S))

Jiji, Kiji = sapt.compute_sapt_JK(Ci, Ci, tensor=sapt.chain_dot(Ci.T, S, Pj, S, Ci))

ExchInd20_ba += 4 * np.vdot(CPHFB_ao, Jiji)
ExchInd20_ba -= 2 * np.vdot(CPHFB_ao, sapt.chain_dot(S, Pi, Kij))
ExchInd20_ba -= 2 * np.vdot(CPHFB_ao, sapt.chain_dot(Kij.T, Pi, S))

sapt_printer('Exch-Ind20,r (A->B)', ExchInd20_ba)
ExchInd20r = ExchInd20_ba + ExchInd20_ab

ind_timer.stop()
### End E200 Induction and Exchange-Induction



# 8. Summary table

All the SAPT0-AO interaction energy contributions have been calculated. All that is left to do is to print out the contributions and the total energy, and to compare the results with the SAPT0 corrections calculated directly by Psi4.


In [ ]:
print('SAPT0 Results')
print('-' * 70)
sapt_printer('Exch10 (S^2)', Exch100)
sapt_printer('Elst10', Elst10)
sapt_printer('Disp20', Disp200)
sapt_printer('Exch-Disp20', ExchDisp20)
sapt_printer('Ind20,r', Ind20r)
sapt_printer('Exch-Ind20,r', ExchInd20r)

print('-' * 70)
sapt0 = Exch100 + Elst10 + Disp200 + ExchDisp20 + Ind20r + ExchInd20r
sapt_printer('Total SAPT0', sapt0)

# ==> Compare to Psi4 <==
psi4.set_options({'df_basis_sapt':'aug-cc-pvtz-ri'})
psi4.energy('sapt0')
Eelst = psi4.get_variable('SAPT ELST ENERGY')
Eexch = psi4.get_variable('SAPT EXCH10(S^2) ENERGY')
Eind  = psi4.get_variable('SAPT IND20,R ENERGY')
Eexind  = psi4.get_variable('SAPT EXCH-IND20,R ENERGY')
Edisp  = psi4.get_variable('SAPT DISP20 ENERGY')
Eexdisp  = psi4.get_variable('SAPT EXCH-DISP20 ENERGY')
psi4.compare_values(Eelst, Elst10, 6, 'Elst100')
psi4.compare_values(Eexch, Exch100, 6, 'Exch100(S^2)')
psi4.compare_values(Edisp, Disp200, 6, 'Disp200')
psi4.compare_values(Eexdisp, ExchDisp20, 6, 'Exch-Disp200')
psi4.compare_values(Eind, Ind20r, 6, 'Ind200,r')
psi4.compare_values(Eexind, ExchInd20r, 6, 'Exch-Ind200,r')



## References

1. A paper that first formulated some SAPT0 and SAPT(DFT) corrections in AOs: "Density-functional theory-symmetry-adapted intermolecular perturbation theory with density fitting: A new efficient method to study intermolecular interaction energies"
	> [[Hesselmann:2005](http://aip.scitation.org/doi/abs/10.1063/1.1824898)] A. Heßelmann, G. Jansen, and M. Schütz, *J. Chem. Phys.* **122**, 014103 (2005)
2. Introduction of the UHF-based open-shell SAPT (mostly in AOs): "Symmetry-adapted perturbation theory based on unrestricted Kohn-Sham orbitals for high-spin open-shell van der Waals complexes"
	> [[Hapka:2012](http://aip.scitation.org/doi/10.1063/1.4758455)] M. Hapka, P. S. Żuchowski, M. M. Szczęśniak, and G. Chałasiński, *J. Chem. Phys.* **137**, 164104 (2012)
3. A new efficient implementation of SAPT(UHF) (and more AO formulas): "Density-fitted open-shell symmetry-adapted perturbation theory and application to π-stacking in benzene dimer cation and ionized DNA base pair steps"
	> [[Gonthier:2016](http://aip.scitation.org/doi/10.1063/1.4963385)] J. F. Gonthier and C. D. Sherrill, *J. Chem. Phys.* **145**, 134106  (2016)
4. The density-matrix formalism for SAPT exchange corrections employed in this work: "Many‐body theory of exchange effects in intermolecular interactions. Density matrix approach and applications to He–F$^−$, He–HF, H$_2$–HF, and Ar–H$_2$ dimers"
	> [[Moszynski:1994b](http://aip.scitation.org/doi/abs/10.1063/1.467225)] R. Moszynski, B. Jeziorski, S. Rybak, K. Szalewicz, and H. L. Williams, *J. Chem. Phys.* **100**, 5080 (1994)
